# **European journal publications BERTopic**

In [1]:
import torch

c:\Users\user\anaconda3\envs\bertopic_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0)) 

Is CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score
import re

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

In [5]:
import os
os.getcwd()

'c:\\Users\\user\\OneDrive - TUM\\1. 연구\\In Progress\\Sci-tech semantic proximity and regional competitiveness\\Data\\Queries\\Sci-Tech Similarity'

In [6]:
import pandas as pd
pub_publication = pd.read_csv('eu_pub_prox_publication.csv')
pub_publication.head()

C:\Users\user\AppData\Local\Temp\ipykernel_60184\2166497544.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pub_publication = pd.read_csv('eu_pub_prox_publication.csv')


,pubid,pubyear,abstract,itemtitle,source
0,2.0,1992,\r\n<p>Applications to release genetically mod...,Environmental risks from the release of geneti...,MOLECULAR ECOLOGY
1,3.0,1992,\r\n<p>A vector system was developed employing...,The construction of recA-deficient Rhizobium m...,MOLECULAR ECOLOGY
2,4.0,1992,\r\n<p>The use of DNA fingerprinting to identi...,DNA fingerprinting in clonal organisms,MOLECULAR ECOLOGY
3,5.0,1992,\r\n<p>Mitochondrial DNA (mtDNA) from 25 blue ...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY
4,7.0,1992,\r\n<p>Males of the bushcricket Poecilimon vel...,Last-male sperm precedence in the bushcricket ...,MOLECULAR ECOLOGY


In [7]:
# check the number of unique pubid in each pubyear
pub_publication.groupby('pubyear')['pubid'].nunique()

pubyear
1980     48454
1981     52298
1982     56122
1983     62820
1984     67034
1985     71256
1986     73766
1987     77961
1988     83830
1989     88437
1990     94818
1991    103016
1992     73907
1993       986
1994      1023
1995       795
1996      1155
1997     34417
1998    271457
1999    277423
2000    281163
2001    283183
2002    284335
2003    291954
2004    301536
2005    312048
2006    328168
2007    349233
2008    364407
2009    372445
2010    386176
2011    453469
2012    472901
2013    496512
2014    500520
2015    514727
2016    530416
2017    540196
2018    551802
2019    569131
2020    621721
2021    677744
2022      8843
Name: pubid, dtype: int64

In [8]:
import pandas as pd
pub_institution = pd.read_csv('eu_pub_prox_institution.csv')
pub_institution.head()

C:\Users\user\AppData\Local\Temp\ipykernel_60184\1327335224.py:2: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  pub_institution = pd.read_csv('eu_pub_prox_institution.csv')


,pubid,pubyear,institution_id_te,organization,country,eu_nuts_id
0,2.0,1992,81734973.0,UNIV YORK,ENGLAND,UKE21
1,2.0,1992,81889854.0,UNIVERSITY OF YORK - UK,ENGLAND,UKE21
2,3.0,1992,82583283.0,UNIVERSITY OF BIELEFELD,GERMANY,DEA41
3,3.0,1992,68681333.0,UNIV BIELEFELD,GERMANY,DEA41
4,4.0,1992,76491407.0,UNIV NOTTINGHAM,ENGLAND,UKF14


In [9]:
# check the number of unique pubid in each pubyear
pub_institution.groupby('pubyear')['pubid'].nunique()

pubyear
1980     48454
1981     52298
1982     56122
1983     62820
1984     67034
1985     71256
1986     73766
1987     77961
1988     83830
1989     88437
1990     94818
1991    103016
1992     73907
1993       986
1994      1023
1995       795
1996      1155
1997     34417
1998    271457
1999    277423
2000    281163
2001    283183
2002    284335
2003    291954
2004    301536
2005    312048
2006    328168
2007    349233
2008    364407
2009    372445
2010    386176
2011    453469
2012    472901
2013    496512
2014    500520
2015    514727
2016    530416
2017    540196
2018    551802
2019    569131
2020    621721
2021    677744
2022      8843
Name: pubid, dtype: int64

In [10]:
pub_institution = pub_institution[['pubid', 'eu_nuts_id']]
pub_institution = pub_institution.drop_duplicates()
pub_institution = pub_institution.dropna() 
pub_institution.head()

,pubid,eu_nuts_id
0,2.0,UKE21
2,3.0,DEA41
4,4.0,UKF14
8,5.0,FR714
14,7.0,DE252


In [11]:
# merge publication and institution on pubid 
df = pd.merge(pub_publication, pub_institution, on='pubid', how = 'outer')
df.head()

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id
0,2.0,1992,\r\n<p>Applications to release genetically mod...,Environmental risks from the release of geneti...,MOLECULAR ECOLOGY,UKE21
1,3.0,1992,\r\n<p>A vector system was developed employing...,The construction of recA-deficient Rhizobium m...,MOLECULAR ECOLOGY,DEA41
2,4.0,1992,\r\n<p>The use of DNA fingerprinting to identi...,DNA fingerprinting in clonal organisms,MOLECULAR ECOLOGY,UKF14
3,5.0,1992,\r\n<p>Mitochondrial DNA (mtDNA) from 25 blue ...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR714
4,7.0,1992,\r\n<p>Males of the bushcricket Poecilimon vel...,Last-male sperm precedence in the bushcricket ...,MOLECULAR ECOLOGY,DE252


In [12]:
# check the lowest and highest pubyear 
df['pubyear'].min(), df['pubyear'].max()

(1980, 2022)

In [13]:
# check how many unique pubid there are in year 2017
df[df['pubyear'] == 2016]['pubid'].nunique()

530416

In [14]:
# check the number of unique pubid in each year
df.groupby('pubyear')['pubid'].nunique()

pubyear
1980     48454
1981     52298
1982     56122
1983     62820
1984     67034
1985     71256
1986     73766
1987     77961
1988     83830
1989     88437
1990     94818
1991    103016
1992     73907
1993       986
1994      1023
1995       795
1996      1155
1997     34417
1998    271457
1999    277423
2000    281163
2001    283183
2002    284335
2003    291954
2004    301536
2005    312048
2006    328168
2007    349233
2008    364407
2009    372445
2010    386176
2011    453469
2012    472901
2013    496512
2014    500520
2015    514727
2016    530416
2017    540196
2018    551802
2019    569131
2020    621721
2021    677744
2022      8843
Name: pubid, dtype: int64

In [15]:
# slice the data to only include data from 1980 to 2021
df = df[(df['pubyear'] >= 1980) & (df['pubyear'] <= 2021)]
df['pubyear'].unique()

array([1992, 1996, 1997, 1998, 1999, 2000, 2001, 1995, 2002, 2003, 2004,
       2006, 1994, 2005, 1989, 1986, 1993, 1990, 1987, 1988, 1991, 1985,
       2007, 1981, 2008, 1982, 1984, 1983, 2009, 1980, 2010, 2012, 2011,
       2013, 2015, 2014, 2017, 2016, 2018, 2019, 2020, 2021], dtype=int64)

In [16]:
# create 'period' column 
def assign_period(pubyear):
    if 1981 <= pubyear <= 1985:
        return 1
    elif 1986 <= pubyear <= 1990:
        return 2 
    elif 1991 <= pubyear <= 1995:
        return 3 
    elif 1996 <= pubyear <= 2000:
        return 4
    elif 2001 <= pubyear <= 2005:
        return 5
    elif 2006 <= pubyear <= 2010:
        return 6
    elif 2011 <= pubyear <= 2015:
        return 7
    elif 2016 <= pubyear <= 2020:
        return 8
    else:
        return 0 
    
df['period'] = df['pubyear'].apply(assign_period)
df.head()

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id,period
0,2.0,1992,\r\n<p>Applications to release genetically mod...,Environmental risks from the release of geneti...,MOLECULAR ECOLOGY,UKE21,3
1,3.0,1992,\r\n<p>A vector system was developed employing...,The construction of recA-deficient Rhizobium m...,MOLECULAR ECOLOGY,DEA41,3
2,4.0,1992,\r\n<p>The use of DNA fingerprinting to identi...,DNA fingerprinting in clonal organisms,MOLECULAR ECOLOGY,UKF14,3
3,5.0,1992,\r\n<p>Mitochondrial DNA (mtDNA) from 25 blue ...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR714,3
4,7.0,1992,\r\n<p>Males of the bushcricket Poecilimon vel...,Last-male sperm precedence in the bushcricket ...,MOLECULAR ECOLOGY,DE252,3


In [17]:
# remove '\r\n<p>' from the abstracts 
df['abstract'] = df['abstract'].str.replace('\r\n<p>', '')
df.head()

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id,period
0,2.0,1992,Applications to release genetically modified o...,Environmental risks from the release of geneti...,MOLECULAR ECOLOGY,UKE21,3
1,3.0,1992,A vector system was developed employing the re...,The construction of recA-deficient Rhizobium m...,MOLECULAR ECOLOGY,DEA41,3
2,4.0,1992,The use of DNA fingerprinting to identify memb...,DNA fingerprinting in clonal organisms,MOLECULAR ECOLOGY,UKF14,3
3,5.0,1992,Mitochondrial DNA (mtDNA) from 25 blue tits Pa...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR714,3
4,7.0,1992,Males of the bushcricket Poecilimon veluchianu...,Last-male sperm precedence in the bushcricket ...,MOLECULAR ECOLOGY,DE252,3


# **특정 기준 이상 논문 보유한 지역, 기간 선정**

In [18]:
# group by eu_nuts_id and period and count the number of unique pubid 
df_grouped = df.groupby(['eu_nuts_id', 'period'])['pubid'].nunique().reset_index()
df_grouped.head()

,eu_nuts_id,period,pubid
0,AT111,4,2
1,AT111,5,4
2,AT112,4,24
3,AT112,5,39
4,AT112,6,2


In [19]:
df_grouped.describe()

,period,pubid
count,6240.000000,6240.000000
mean,4.799519,594.360577
std,1.643504,2264.445968
min,0.000000,1.000000
25%,4.000000,4.000000
50%,5.000000,23.000000
75%,6.000000,165.000000
max,8.000000,51695.000000


In [20]:
# drop rows where pubid is less than 50 
df_grouped = df_grouped[df_grouped['pubid'] >= 50]
df_grouped

,eu_nuts_id,period,pubid
9,AT121,5,55
13,AT122,5,107
16,AT123,4,59
17,AT123,5,101
21,AT124,5,77
...,...,...,...
6232,UKN04,4,414
6233,UKN04,5,838
6234,UKN04,6,86
6236,UKN05,4,146


In [21]:
df_grouped['eu_nuts_id'].nunique()

987

In [22]:
df_grouped.to_csv('pub_above_50.csv', index=False)

In [23]:
# drop rows in df where each pair of eu_nuts_id and period is not in df_grouped
df = df.merge(df_grouped[['eu_nuts_id', 'period']], on=['eu_nuts_id', 'period'], how='inner')
df

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id,period
0,5.0,1992,Mitochondrial DNA (mtDNA) from 25 blue tits Pa...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR714,3
1,7135508.0,1992,In the interpretation of the Antarctic deep ic...,Evidence for an early Holocene climatic optimu...,CLIMATE DYNAMICS,FR714,3
2,7136421.0,1994,"In this letter, the Fourier transform of the o...",Frequency Domain Volterra Filters in Terms of ...,IEEE SIGNAL PROCESSING LETTERS,FR714,3
3,7145640.0,1991,The objectives of these studies (1) were: (a) ...,Interactions between pectins and multivalent c...,FOOD HYDROCOLLOIDS,FR714,3
4,7145654.0,1991,NaN,Study of the autohydrolysis of Rubus fruticosu...,FOOD HYDROCOLLOIDS,FR714,3
...,...,...,...,...,...,...,...
3666663,10544850.0,2006,This paper deals with two topics: the generati...,Incoherent pair generation in a beam-beam inte...,PHYSICAL REVIEW SPECIAL TOPICS-ACCELERATORS AN...,DE406,6
3666664,10587185.0,2006,The first measurements of double-hadron produc...,Double-hadron leptoproduction in the nuclear m...,PHYSICAL REVIEW LETTERS,DE406,6
3666665,10592724.0,2006,Harmonic sums and their generalizations are ex...,XSUMMER - Transcendental functions and symboli...,COMPUTER PHYSICS COMMUNICATIONS,DE406,6
3666666,10608505.0,2006,Background: Therapeutic options for patients w...,Aripiprazole augmentation of clozapine in trea...,CLINICAL DRUG INVESTIGATION,DE406,6


In [24]:
# customize stop words list
custom_stop_words = ["determining", "method", "includes", "based", "second", "using", 'showed', 'problem', "study", "non"
                     "12", "14", "15", "2014", "iii", "ii", "10", "1a", "11", "cm", "60", '28', '250', '17',
                     "25", "40", "model", "2005", "ar", "rh", "kv", "sb", "ac", "www", "nm", 'srr', 'xy', 'sigma',
                     "mo", "nm", "18", "ee", "pb", "gev", "mu", "rh", "ln", "a15", "13", "ag", '55', 'effect','nd3',
                     "nm", "qds", "iv", "ag", "rms", "le", "p53", "pl", "fe", "ir", "4abn", "nmr", "ni", '05', '74',
                     "ta", "tio2"< "thz", "db", "hz", "iso", "pss", "1310", "nu", "tf", "tf", '33', '350',
                     "km", "tev", "gev", "fb", "tl", "t2dm", "nlc", "dot", "si", "cb7", "mc", "soi", '100', 
                     "dots", "pi", "qd", "la", "cdw", "iqp", "ccl4", "no2", "mm", "p1", "bi", "ho", "rs", '00',
                     "qh", "tio2", "nu", "nir", "time", "bm12", "ssi", "u6", "current", "nmr", 'hs', 'sp',
                     "gev", "tot", "1h", "hh", "hi", "qds", "pss", "no2", "zb", "wz", "al", "ge", 'ff', 'der',
                     "ect", "mdd", "95", "girls", "uc", "cpe", "di", "uc", "er3", "new", 's190', 's208', '44',
                     "od", "ch3", "yb", "eu", "tb", "nr", "ope3", "4tp", "mu", "center", "la", '3b', 'eag1',
                     "number", "su", "ch4", "fe", "cr", "ni", "eat", "ws", "spp", "mn2", "mm", 'ss', 'oc',
                     "rf", "li", "2016", "16", "sic", "si", "von", "sept3", "0d", "36", 'igf1r', 'ma', 'mnc', ''
                     'o3', 'correctly', 'tb', 'te', 'il', 'cd39', 'cd73', 'au', 'ou', 'ml', 'mg', 'puo2', '298',
                     'sec', 'eta', 'ba122', 'al', 'cgm', 'h2s', 'ir', 'sni', '45', '75', 'tio2', '60', 'sp', '4000', 
                     'ge', 'si', '5p', '120', '2d', 'random', 'theory', 'li', 'qds', 'qd', 'p3ht', 'mu', '69', '71',
                     'p2', 'gw', '511', '3885', 'yb3', '25', 'sir', '240', 'ro', 't34', '10', '6p', 'bbb', 'c1', 'c3',
                     'nno', 'nlo', 'ag', 'sb', 'naf', 'ga', 'se', 'cu', 'gd', 'pf6', 'dots', 'dat', '4f', 
                     'ch3', 'center', 'centre', 'mc3t3', 'e1', '1064', 'jc', 'ga', 'cf3', 'bu', 'zn', 'near', 
                     'np', 'zn', 'cp', 'fr', 'nmc', 'nca', 'oco', 'dsb', 'ssb', 'oe', 'mn', '300', '80', 'b12x12',
                     'nh', 'br', 'mw', '77', 'ig', '24', 'sigma', '700', 'ij', '1h', 'approach', '4b', '343', 
                     's6', 'ce3', 'advantages', 'phase', '001', 'ad', 'dy3', '111', '2000', 'matter', '99',
                     'results', 'used', 'methods', 'analysis', 'used', 'associated', 'different', 'argue', 'article', 
                     'let', 'research', 'data', 'paper', 'examines', 'prove', 'proposed', 'samples', 'study', 'similar', 
                     'compared', 'significantly', 'high', 'publication', 'significant', 'use', 'studies', 'category', 
                     'use', 'field', 'group', 'years', '000', '2024', 'field', 'risk', 'non', 'age'
                     ]  

# 기본 영어 stop words (서술어, 조사 등) 목록 가져오기
default_stop_words = text.ENGLISH_STOP_WORDS

# 기본 영어 stop words에 추가할 단어들 합치기
all_stop_words = list(default_stop_words.union(custom_stop_words))

In [25]:
df.head()

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id,period
0,5.0,1992,Mitochondrial DNA (mtDNA) from 25 blue tits Pa...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR714,3
1,7135508.0,1992,In the interpretation of the Antarctic deep ic...,Evidence for an early Holocene climatic optimu...,CLIMATE DYNAMICS,FR714,3
2,7136421.0,1994,"In this letter, the Fourier transform of the o...",Frequency Domain Volterra Filters in Terms of ...,IEEE SIGNAL PROCESSING LETTERS,FR714,3
3,7145640.0,1991,The objectives of these studies (1) were: (a) ...,Interactions between pectins and multivalent c...,FOOD HYDROCOLLOIDS,FR714,3
4,7145654.0,1991,NaN,Study of the autohydrolysis of Rubus fruticosu...,FOOD HYDROCOLLOIDS,FR714,3


In [ ]:
import pandas as pd
from tqdm import tqdm
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from bertopic.vectorizers import ClassTfidfTransformer

# Create an empty list to store the results
all_results = []

# Iterate over each 'eu_nuts_id' and 'period', showing progress with tqdm
for nuts in tqdm(df['eu_nuts_id'].unique(), desc="NUTS regions"):
    for i in range(1, 9):
        # Create a filtered copy of the dataset for 'eu_nuts_id' and 'period'
        filtered_df = df[(df['eu_nuts_id'] == nuts) & (df['period'] == i)].copy()

        # Extract the 'abstract' column for BERTopic modeling, and ensure all entries are strings
        docs = filtered_df['abstract'].dropna().astype(str).tolist()

        if len(docs) == 0:
            print(f"No valid documents for NUTS region {nuts} and period {i}")
            continue

        try:
            # Term Frequency, Inverse Document Frequency & transform docs into vectors
            vectorizer_model = CountVectorizer(stop_words=all_stop_words)
            ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
            embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
            embeddings = embedding_model.encode(docs, show_progress_bar=True)

            # Dynamically set the number of neighbors for UMAP based on the number of documents
            n_neighbors = min(30, len(docs) - 1)  # Ensures n_neighbors <= number of documents 15 or 30
            if n_neighbors <= 2:  # UMAP requires n_neighbors to be at least 2
                print(f"Skipping NUTS region {nuts} and period {i} due to insufficient documents.")
                continue

            umap_model = UMAP(n_neighbors=n_neighbors, min_dist=0.1, metric='cosine') # 0.0-0.3, a lower min_dist makes the clusters tighgter, whereas a higher value makes dispersed clusters

            # hyper parameter 설정, 최대 10개의 주제를, 최소 min_topic_sizes 보다 큰 클러스터로 생성 
            n_gram_ranges = (1, 2)
            min_topic_sizes = max(2, int(len(docs) * 0.01))  # 한 클러스터에 포함되는 최소 문서 수 (1, 1.5, 2, 3%)
            nr_topics_options = 40  # 최대 클러스터링 수 (5, 10, 15, 20개)

            # 문서별로 할당된 주제 topics와 확률 probs
            topic_model = BERTopic(
                n_gram_range=n_gram_ranges,
                min_topic_size=min_topic_sizes,
                nr_topics=nr_topics_options,
                embedding_model=embedding_model,
                vectorizer_model=vectorizer_model,
                calculate_probabilities=True,
                ctfidf_model=ctfidf_model,
                umap_model=umap_model
            )

            topics, probs = topic_model.fit_transform(docs, embeddings)

            # 결과를 저장
            freq = topic_model.get_topic_info()

            # Add 'eu_nuts_id' and 'period' columns to the topic info DataFrame
            freq['eu_nuts_id'] = nuts
            freq['period'] = i

            # Append the result to the list
            all_results.append(freq)

            print(f"Processed topic information for NUTS region {nuts} and period {i}")

        except Exception as e:
            print(f"Error processing NUTS region {nuts} and period {i}: {e}")

# Combine all the results into a single DataFrame
final_results = pd.concat(all_results, ignore_index=True)

# Save the final combined DataFrame to a single CSV file
final_results.to_csv('pub_bertopic.csv', index=False)

NUTS regions:   0%|          | 0/987 [00:00<?, ?it/s]

No valid documents for NUTS region FR714 and period 1
No valid documents for NUTS region FR714 and period 2


Batches: 100%|██████████| 3/3 [00:03<00:00,  1.24s/it]


Processed topic information for NUTS region FR714 and period 3


Batches: 100%|██████████| 316/316 [00:17<00:00, 18.07it/s]


...

Processed topic information for NUTS region ITI13 and period 5
No valid documents for NUTS region ITI13 and period 6
No valid documents for NUTS region FI1C4 and period 1
No valid documents for NUTS region FI1C4 and period 2
No valid documents for NUTS region FI1C4 and period 3
No valid documents for NUTS region FI1C4 and period 4


NUTS regions: 100%|█████████▉| 986/987 [3:41:11<00:02,  2.41s/it]

Processed topic information for NUTS region FI1C4 and period 5
No valid documents for NUTS region FI1C4 and period 6
No valid documents for NUTS region EL651 and period 1
No valid documents for NUTS region EL651 and period 2
No valid documents for NUTS region EL651 and period 3
No valid documents for NUTS region EL651 and period 4


NUTS regions: 100%|██████████| 987/987 [3:41:14<00:00, 13.45s/it]

Processed topic information for NUTS region EL651 and period 5
No valid documents for NUTS region EL651 and period 6


In [27]:
final_results[final_results['Topic'] == -1]

,Topic,Count,Name,Representation,Representative_Docs,eu_nuts_id,period
4,-1,704,-1_gan_growth_hcv_hepatitis,"[gan, growth, hcv, hepatitis, control, populat...",[A model is presented in order to explain most...,FR714,4
8,-1,1025,-1_gan_bone_nanotubes_diamond,"[gan, bone, nanotubes, diamond, aln, polariton...",[Three-dimensional numerical simulations of ul...,FR714,5
14,-1,533,-1_species_elsevier_reserved_rights,"[species, elsevier, reserved, rights, process,...",[The formation of nanoscopic InGaAs ring struc...,FR714,6
35,-1,539,-1_sperm_dna_dogs_degrees,"[sperm, dna, dogs, degrees, temperature, water...",[A computer-based lattice model of the step gr...,UKN01,4
53,-1,747,-1_temperature_concrete_dielectric_water,"[temperature, concrete, dielectric, water, dry...",[The limited availability of published physica...,UKN01,5
...,...,...,...,...,...,...,...
38907,-1,12,-1_meja_varicocele_produced_grain,"[meja, varicocele, produced, grain, left, hirs...",[Treatment of cut freesia var. Cote d'Azur flo...,EL653,5
38914,-1,10,-1_gt_silicon_vitamin_lpcvd,"[gt, silicon, vitamin, lpcvd, diamine, mol, ps...",[In this paper the kinetic parameters depictin...,RO313,5
38921,-1,14,-1_bdd_band_ocd_los,"[bdd, band, ocd, los, sepsis, lap, survival, m...",[Background and aims of the study: The study a...,ITI13,5
38937,-1,1,-1_children_hypoglycaemia_severe_neuropsycholo...,"[children, hypoglycaemia, severe, neuropsychol...",[Previous studies have shown that recurrent se...,FI1C4,5


In [28]:
# Step 1: Total count per eu_nuts_id and period
total_count = final_results.groupby(['eu_nuts_id', 'period'])['Count'].sum()

# Step 2: Count for Topic == -1 per eu_nuts_id and period
minus_one_count = final_results[final_results['Topic'] == -1].groupby(['eu_nuts_id', 'period'])['Count'].sum()

# Step 3: Calculate ratio
ratio = (minus_one_count / total_count).reset_index(name='topic_minus_one_ratio')

# Optional: rename columns for clarity
ratio = ratio.rename(columns={'eu_nuts_id': 'NUTS', 'period': 'Period'})
ratio

,NUTS,Period,topic_minus_one_ratio
0,AT121,5,0.120000
1,AT122,5,0.069307
2,AT123,4,NaN
3,AT123,5,0.125000
4,AT124,5,0.121622
...,...,...,...
2290,UKN04,4,0.191710
2291,UKN04,5,0.159705
2292,UKN04,6,0.244186
2293,UKN05,4,0.048780


In [29]:
# check rows where eu_nuts_id = UKM26 and period = 4
final_results[(final_results['eu_nuts_id'] == 'AT121') & (final_results['period'] == 5)]

,Topic,Count,Name,Representation,Representative_Docs,eu_nuts_id,period
37728,-1,6,-1_leaf_litter_leaves_employment,"[leaf, litter, leaves, employment, reach, stre...",[Coarse (0.5 mm) and fine (0.1 mm) mesh size b...,AT121,5
37729,0,6,0_colonies_dca_aggregations_drones,"[colonies, dca, aggregations, drones, drone, a...",[Sacbrood virus (SBV) infects Larvae of the ho...,AT121,5
37730,1,5,1_hrqol_milk_mould_raw,"[hrqol, milk, mould, raw, statistical, demogra...",[Monitoring performance is essential for the e...,AT121,5
37731,2,10,2_kynurenine_control_brain_acid,"[kynurenine, control, brain, acid, levels, pat...",[Background Eosinophil activation is character...,AT121,5
37732,3,3,3_steers_heifers_feeding_silage,"[steers, heifers, feeding, silage, bulls, corn...","[The effect of ration, sex and slaughter weigh...",AT121,5
37733,4,2,4_working_work_hours_standard,"[working, work, hours, standard, farm, taken, ...",[To help local regulators mitigate non-point s...,AT121,5
37734,5,9,5_soil_gully_yield_losses,"[soil, gully, yield, losses, crop, compaction,...",[Soil phosphorus tests offer a potentially pow...,AT121,5
37735,6,6,6_river_sediment_flow_basin,"[river, sediment, flow, basin, danube, nutrien...",[We investigated interstitial flow velocities ...,AT121,5
37736,7,3,7_chemistry_variation_no3_lakes,"[chemistry, variation, no3, lakes, water, ripa...",[The water balance for the site Muhleggerkopfl...,AT121,5


In [32]:
ratio['topic_minus_one_ratio'].describe()

count    2086.000000
mean        0.160050
std         0.087084
min         0.000041
25%         0.096507
50%         0.161322
75%         0.217965
max         0.484416
Name: topic_minus_one_ratio, dtype: float64